In [1]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Agent Engine in Vertex AI - orchestration

This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,langchain,ag2]

In [3]:
# Note:  Need to restart the kernel
%pip install  --user --quiet google-cloud-firestore
%pip install  --user --quiet langchain-google-firestore
%pip install  --user --quiet langchain-google-vertexai

In [4]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
#MODEL_NAME = "gemini-1.5-flash-002"
MODEL_NAME = "gemini-2.0-flash-001"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Configuration and initialization

In [5]:
# # Static configuration
# PROJECT_ID = "ai-hangsik"
# LOCATION = "us-central1"
# STAGING_BUCKET = "gs://reasoning_7424"
# MODEL = "gemini-1.5-flash"

# Firestore
# DATABASE="chat-history"
# COLLECTION="history"

### Initialize Vertex AI

In [6]:
# @title Create a bucket.
BUCKET_URI = f"gs://agent-0319"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0319/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0319' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [7]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [8]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://agent-0319/
No changes made to gs://agent-0319/


In [9]:
import vertexai
# from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

# custom_prompt_template = {
#     "user_input": lambda x: x["input"],
#     "history": lambda x: x["history"],
#     "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
# } | ChatPromptTemplate.from_messages([
#     ("placeholder", "{history}"),
#     ("user", "{user_input}"),
#     ("placeholder", "{agent_scratchpad}"),
# ])

In [11]:
from vertexai.preview.reasoning_engines import LanggraphAgent

def langgraph_builder(*, model, **kwargs):
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langgraph.graph import END, MessageGraph

    output_parser = StrOutputParser()

    planner = ChatPromptTemplate.from_template(
        "Generate an argument about: {input}"
    ) | model | output_parser

    pros = ChatPromptTemplate.from_template(
        "List the positive aspects of {input}"
    ) | model | output_parser

    cons = ChatPromptTemplate.from_template(
        "List the negative aspects of {input}"
    ) | model | output_parser

    summary = ChatPromptTemplate.from_template(
        "Input:{input}\nGenerate a final response given the critique",
    ) | model | output_parser

    builder = MessageGraph()

    builder.add_node("planner", planner)
    builder.add_node("pros", pros)
    builder.add_node("cons", cons)
    builder.add_node("summary", summary)

    builder.add_edge("planner", "pros")
    builder.add_edge("planner", "cons")
    builder.add_edge("pros", "summary")
    builder.add_edge("cons", "summary")
    builder.add_edge("summary", END)
    builder.set_entry_point("planner")

    return builder.compile()


In [12]:
from vertexai.generative_models import grounding, Tool

grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

In [15]:
from vertexai.preview.reasoning_engines import LangchainAgent

agent = LanggraphAgent(
    model=MODEL_NAME,
    tools=[grounded_search_tool],
    # prompt=custom_prompt_template,
    runnable_builder=langgraph_builder,
)


In [16]:
from time import perf_counter

query = "생성형 AI의 대해서 설명해주세요."

t1_start = perf_counter()
response = agent.query(input={"role": "user", "content": query})
t1_end = perf_counter()

display(Markdown(f"Elapse time : {t1_end-t1_start}"))

for re in response:
  display(Markdown((re['kwargs']['content'])))
  print('======')

Elapse time : 9.848816120000265

생성형 AI의 대해서 설명해주세요.

## 생성형 AI: 혁신적인 가능성과 잠재적 위험의 양날의 검

생성형 AI는 텍스트, 이미지, 오디오, 비디오 등 다양한 형태의 콘텐츠를 스스로 '창작'해낼 수 있는 인공지능 기술입니다. 이는 기존의 AI가 특정 작업을 수행하거나 데이터를 분석하는 데 집중했던 것과 달리, 데이터의 패턴을 학습하여 새로운 것을 만들어낸다는 점에서 획기적인 진보를 보여줍니다.

**주요 강점 및 기회:**

*   **창의성 및 생산성 향상:** 생성형 AI는 콘텐츠 제작 과정을 자동화하고 아이디어를 발상하는 데 도움을 주어 인간의 창의성을 증진하고 생산성을 극대화할 수 있습니다. 예를 들어, 마케팅 문구 생성, 디자인 시안 제작, 음악 작곡 등 다양한 분야에서 활용될 수 있습니다.
*   **개인 맞춤형 경험 제공:** 사용자의 선호도와 요구에 맞춰 개인화된 콘텐츠를 생성함으로써 더욱 몰입감 있는 경험을 제공할 수 있습니다. 예를 들어, 개인의 취향에 맞는 맞춤형 뉴스 기사, 학습 자료, 게임 콘텐츠 등을 제공할 수 있습니다.
*   **새로운 비즈니스 모델 창출:** 생성형 AI는 완전히 새로운 제품과 서비스를 개발하고 기존 비즈니스 모델을 혁신하는 데 기여할 수 있습니다. 예를 들어, AI가 생성한 예술 작품 판매, AI 기반의 콘텐츠 제작 플랫폼 구축 등의 새로운 사업 기회가 생겨날 수 있습니다.
*   **연구 및 개발 가속화:** 신약 개발, 소재 개발 등 다양한 분야에서 시뮬레이션과 데이터 분석을 통해 연구 개발 과정을 가속화하고 혁신적인 발견을 촉진할 수 있습니다.

**잠재적 위험 및 윤리적 문제:**

*   **허위 정보 및 가짜 콘텐츠 확산:** 생성형 AI는 매우 현실적인 가짜 이미지, 오디오, 비디오 등을 쉽게 만들어낼 수 있어 허위 정보와 가짜 콘텐츠의 확산을 심화시키고 사회적 혼란을 야기할 수 있습니다.
*   **저작권 및 지적 재산권 침해:** 학습 데이터에 사용된 저작권이 있는 콘텐츠를 모방하거나 표절하는 경우가 발생할 수 있으며, AI가 생성한 콘텐츠의 저작권 소유 주체에 대한 논쟁이 발생할 수 있습니다.
*   **일자리 감소 및 경제적 불평등 심화:** 특정 직업군, 특히 콘텐츠 제작과 관련된 일자리가 자동화됨으로써 실업률 증가와 경제적 불평등 심화를 초래할 수 있습니다.
*   **편향성 및 차별 심화:** 학습 데이터에 내재된 편향이 AI 모델에 반영되어 성별, 인종, 사회적 계층 등에 대한 차별적인 콘텐츠를 생성할 수 있습니다.
*   **악용 및 오용 가능성:** 생성형 AI 기술은 사이버 범죄, 테러, 정치적 선전 등 다양한 악의적인 목적으로 사용될 수 있으며, 인간의 통제를 벗어난 위험한 결과를 초래할 수 있습니다.

**결론:**

생성형 AI는 혁신적인 기술이지만, 그 잠재력을 최대한 활용하고 위험을 최소화하기 위해서는 신중한 접근이 필요합니다. 기술 개발과 더불어 윤리적 문제에 대한 심도 있는 논의, 관련 법규 및 규제 마련, 그리고 사회 전체의 교육과 인식 개선이 필수적입니다. 생성형 AI가 인류에게 긍정적인 영향을 미칠 수 있도록 책임감 있는 개발과 활용이 이루어져야 할 것입니다.


제공된 텍스트에 따르면 생성형 AI의 부정적인 측면은 다음과 같습니다.

*   **허위 정보 및 가짜 콘텐츠 확산:** 생성형 AI는 매우 현실적인 가짜 이미지, 오디오, 비디오 등을 쉽게 만들어낼 수 있어 허위 정보와 가짜 콘텐츠의 확산을 심화시키고 사회적 혼란을 야기할 수 있습니다.
*   **저작권 및 지적 재산권 침해:** 학습 데이터에 사용된 저작권이 있는 콘텐츠를 모방하거나 표절하는 경우가 발생할 수 있으며, AI가 생성한 콘텐츠의 저작권 소유 주체에 대한 논쟁이 발생할 수 있습니다.
*   **일자리 감소 및 경제적 불평등 심화:** 특정 직업군, 특히 콘텐츠 제작과 관련된 일자리가 자동화됨으로써 실업률 증가와 경제적 불평등 심화를 초래할 수 있습니다.
*   **편향성 및 차별 심화:** 학습 데이터에 내재된 편향이 AI 모델에 반영되어 성별, 인종, 사회적 계층 등에 대한 차별적인 콘텐츠를 생성할 수 있습니다.
*   **악용 및 오용 가능성:** 생성형 AI 기술은 사이버 범죄, 테러, 정치적 선전 등 다양한 악의적인 목적으로 사용될 수 있으며, 인간의 통제를 벗어난 위험한 결과를 초래할 수 있습니다.

The positive aspects of generative AI are:

*   **Increased creativity and productivity:** It automates content creation and helps with idea generation, enhancing human creativity and maximizing productivity. (e.g., generating marketing copy, designing drafts, composing music)
*   **Personalized experiences:** It can create personalized content based on user preferences, providing more immersive experiences. (e.g., custom news articles, learning materials, game content)
*   **New business model creation:** It can develop entirely new products and services and innovate existing business models. (e.g., selling AI-generated artwork, building AI-based content creation platforms)
*   **Accelerated research and development:** It accelerates research and development in various fields, such as drug and material development, through simulations and data analysis, promoting innovative discoveries.

생성형 AI는 혁신적인 기술이지만, 긍정적인 측면과 함께 심각한 부정적인 측면도 동시에 가지고 있습니다.

**긍정적인 측면:**

*   **창의성 및 생산성 향상:** 콘텐츠 제작 자동화, 아이디어 발상 지원 등을 통해 인간의 창의성을 증진하고 생산성을 극대화합니다. (예: 마케팅 문구 생성, 디자인 시안 제작, 음악 작곡)
*   **개인 맞춤형 경험 제공:** 사용자 선호도에 맞는 콘텐츠를 생성하여 몰입감 있는 경험을 제공합니다. (예: 맞춤형 뉴스, 학습 자료, 게임 콘텐츠)
*   **새로운 비즈니스 모델 창출:** 새로운 제품/서비스 개발 및 기존 비즈니스 모델 혁신에 기여합니다. (예: AI 생성 예술 작품 판매, AI 기반 콘텐츠 제작 플랫폼 구축)
*   **연구 개발 가속화:** 신약 개발, 소재 개발 등 다양한 분야에서 시뮬레이션과 데이터 분석을 통해 혁신적인 발견을 촉진합니다.

**부정적인 측면:**

*   **허위 정보 및 가짜 콘텐츠 확산:** 현실적인 가짜 이미지, 오디오, 비디오 등을 쉽게 생성하여 사회적 혼란을 야기합니다.
*   **저작권 및 지적 재산권 침해:** 학습 데이터의 저작권 침해 및 AI 생성 콘텐츠의 저작권 소유 주체 논쟁을 발생시킵니다.
*   **일자리 감소 및 경제적 불평등 심화:** 콘텐츠 제작 관련 일자리 자동화로 실업률 증가 및 경제적 불평등을 심화시킬 수 있습니다.
*   **편향성 및 차별 심화:** 학습 데이터의 편향이 반영되어 차별적인 콘텐츠를 생성할 수 있습니다.
*   **악용 및 오용 가능성:** 사이버 범죄, 테러, 정치적 선전 등 악의적인 목적으로 사용될 수 있습니다.

따라서 생성형 AI의 잠재력을 최대한 활용하고 위험을 최소화하기 위해서는 윤리적 문제에 대한 심도 있는 논의, 관련 법규 및 규제 마련, 사회 전체의 교육과 인식 개선이 필수적입니다. 책임감 있는 개발과 활용을 통해 생성형 AI가 인류에게 긍정적인 영향을 미칠 수 있도록 노력해야 합니다.
